In [1]:
import random
import numpy as np
import pandas as pd
import duckdb

Autre exemple: assurance santé. <br />
On pourrait vouloir grouper:
- par type de soin
- par type de contrat
- par groupe d'age du bénéficiaire
- et par toutes les combinaisons possibles des groupes ci-dessus:
- par type de soin et groupe d'age
- par type de contrat et groupe d'age
- par type de soin et type de contrat
- par type de soin, contrat, et groupe d'âge

# Data

In [2]:
random.seed(42)
num_samples = 1000

contrats = ["senior", "jeunes", "expat", "famille", "salarié"]
sexe = ["homme", "femme"]
type_acte = {"pharmacie": 15,
"consultation_generaliste": 25,
"hospitalisation": 2800,
"biologie": 150,
"radio": 1300,
"maternite": 1700}
groupe_age = ["18-25", "25-45", "45-65", "65+"]
annee = [2017, 2018, 2019]


# Initialize empty lists to store the data
contrats_data = []
sexe_data = []
type_acte_data = []
groupe_age_data = []
annee_data = []
cost_data = []

# Generate random data for each category
for _ in range(num_samples):
    contrats_data.append(random.choice(contrats))
    sexe_data.append(random.choice(sexe))
    if sexe_data == "femme":
        type_acte_choice = random.choice(list(type_acte.keys()))
    else:
        type_acte_options = list(type_acte.keys())
        type_acte_options.remove("maternite")
        type_acte_choice = random.choice(type_acte_options)
        
    type_acte_data.append(type_acte_choice)
    cost_mean = type_acte[type_acte_choice]
    cost_data.append(np.random.normal(cost_mean, cost_mean // 3.5))  # Assuming a standard deviation of 50 for costs
    groupe_age_data.append(random.choice(groupe_age))
    annee_data.append(random.choice(annee))


In [4]:
# Create a DataFrame to store the dataset
df = pd.DataFrame({
    'type_contrat': contrats_data,
    'sexe': sexe_data,
    'type_acte': type_acte_data,
    'groupe_age': groupe_age_data,
    'annee': annee_data,
    'montant_rembourse': cost_data
})
df

,type_contrat,sexe,type_acte,groupe_age,annee,montant_rembourse
0,senior,homme,hospitalisation,25-45,2017,3247.451445
1,jeunes,homme,radio,18-25,2019,1501.571247
2,famille,homme,pharmacie,18-25,2017,9.558023
3,jeunes,homme,radio,25-45,2019,978.555936
4,salarié,femme,consultation_generaliste,65+,2019,32.544643
...,...,...,...,...,...,...
995,famille,homme,biologie,45-65,2018,212.349955
996,jeunes,homme,hospitalisation,65+,2019,2602.594479
997,salarié,femme,pharmacie,18-25,2017,16.432729
998,expat,homme,hospitalisation,45-65,2019,2670.667271


# Exercices: Rollup

Voici le montant global remboursé par la caisse d'assurance sur la période 2017-2019

In [5]:
query = """
SELECT SUM(montant_rembourse) 
FROM df
"""
duckdb.sql(query)

┌────────────────────────┐
│ sum(montant_rembourse) │
│         double         │
├────────────────────────┤
│       865056.458683528 │
└────────────────────────┘

Votre PO vous demande le montant total remboursé par type de contrat

In [8]:
%load solutions/7groupby_type_contrat.py

┌──────────────┬────────────────────────┐
│ type_contrat │ sum(montant_rembourse) │
│   varchar    │         double         │
├──────────────┼────────────────────────┤
│ senior       │      188512.4940319906 │
│ jeunes       │     167405.76969459036 │
│ famille      │     176500.57874175988 │
│ salarié      │     189877.38475949003 │
│ expat        │     142760.23145569855 │
└──────────────┴────────────────────────┘

Votre Manager vous transmet une autre demande du PO: <br />
il vous demande le montant total remboursé par type de contrat
ET par type d'acte

In [12]:
# %load solutions/8groupby_typecontrat_typeacte.py

┌──────────────┬──────────────────────────┬────────────────────────┐
│ type_contrat │        type_acte         │ sum(montant_rembourse) │
│   varchar    │         varchar          │         double         │
├──────────────┼──────────────────────────┼────────────────────────┤
│ expat        │ consultation_generaliste │      1116.385297254188 │
│ expat        │ pharmacie                │      517.9811981305853 │
│ expat        │ hospitalisation          │      84718.76675721048 │
│ expat        │ radio                    │      49798.37343145399 │
│ expat        │ biologie                 │     6608.7247716492275 │
│ famille      │ consultation_generaliste │      756.0180671379243 │
│ famille      │ pharmacie                │     413.52090620497063 │
│ famille      │ biologie                 │      8334.063536205975 │
│ famille      │ radio                    │      59854.03780717285 │
│ famille      │ hospitalisation          │     107142.93842503817 │
│    ·         │        ·         

Le PO revient en râlant: le manager n'avait rien compris ! <br />
Il veut le montant total remboursé par type de contrat <br />
ET le montant total remboursé par type d'acte

Hint: commencez simple, faites le avec un Union

In [15]:
%load solutions/9groupby_unions.py

┌──────────────────────────┬────────────────────────┐
│        typologie         │ sum(montant_rembourse) │
│         varchar          │         double         │
├──────────────────────────┼────────────────────────┤
│ hospitalisation          │      557117.0437531122 │
│ radio                    │      266648.2365038529 │
│ pharmacie                │     2577.0978960946113 │
│ consultation_generaliste │     5122.2520484052375 │
│ biologie                 │      33591.82848206439 │
│ senior                   │      188512.4940319906 │
│ jeunes                   │     167405.76969459036 │
│ famille                  │     176500.57874175988 │
│ salarié                  │     189877.38475949003 │
│ expat                    │     142760.23145569855 │
├──────────────────────────┴────────────────────────┤
│ 10 rows                                 2 columns │
└───────────────────────────────────────────────────┘

Votre tech lead vient vous voir:

<blockquote> Ton code fait le taff, mais j'ai récemment lu un article sur les GROUPING SETS, je pense que ça permettrait de simplifier le code sur ton problème, tu peux implémenter ça ? Merci ! </blockquote>

In [ ]:
%load solutions/10groupby_groupingsets.py

Votre manager revient à la charge:
<blockquote> Perso, je trouve que c'était très bien d'avoir aussi le montant total remboursé par type de contrat
ET par type d'acte. J'ai discuté avec le PO et il est OK. Garde ce que t'as fait, mais remets aussi les subdivisions </blockquote>

Vous décidez d'utiliser ROLLUP pour obtenir tout ça facilement:

In [ ]:
%load solutions/11rollup.py

# Exercices: CUBE

Votre tech lead vérifie votre code et vous dit:

<blockquote> Tu t'es planté. ROLLUP ça enlève progressivement le niveau de regroupement le plus à droite de ta liste. Résultat: on n'a pas les sommes par type_acte uniquement (il manque 5 lignes) 
Il faut trouver une autre solution ! </blockquote>


Il nous faut donc:
- la somme par type d'acte (5 lignes),
- la somme par type de contrat (5 lignes),
- ET la somme par type d'acte + type de contrat (25 lignes),

Et effectivement, avec ROLLUP il nous manquait la somme par type d'acte uniquement (5 lignes)

Vous décidez d'utiliser un CUBE Pour avoir toutes les options:

In [20]:
%load solutions/12cube.py

,type_contrat,type_acte,sum(montant_rembourse)
0,expat,biologie,6608.724772
1,expat,consultation_generaliste,1116.385297
2,expat,hospitalisation,84718.766757
3,expat,pharmacie,517.981198
4,expat,radio,49798.373431
5,expat,NaN,142760.231456
6,famille,biologie,8334.063536
7,famille,consultation_generaliste,756.018067
8,famille,hospitalisation,107142.938425
9,famille,pharmacie,413.520906


# Grand' Final

En lisant le rapport fourni par votre P.O. sur la base de vos chiffres <br />
Votre N+2 a eu plein d'idées.

<img src="images/asterix_plein_idees.gif" />
<p style="text-align:center"><i>Le N+2</i></p>

<blockquote> Je veux la somme des montants remboursés par:
 type_contrat / type_acte / groupe_age / sexe / annee
</blockquote>

et je veux avoir les chiffres globaux pour chacune de ces sous-catégories:
- type_contrat / type_acte / groupe_age / sexe /
- type_contrat / type_acte / groupe_age /
- type_contrat / type_acte / 
- type_contrat / 

Maintenant que vous connaissez ROLLUP, ça devrait être un jeu d'enfant:

In [23]:
%load solutions/13rollup_finale.py

,type_contrat,type_acte,groupe_age,sexe,annee,sum(montant_rembourse)
0,expat,biologie,18-25,femme,2017.0,150.741046
1,expat,biologie,18-25,femme,2018.0,82.941262
2,expat,biologie,18-25,femme,2019.0,172.829226
3,expat,biologie,18-25,femme,NaN,406.511534
4,expat,biologie,18-25,homme,2017.0,125.018416
5,expat,biologie,18-25,homme,2018.0,210.347489
6,expat,biologie,18-25,homme,2019.0,496.769451
7,expat,biologie,18-25,homme,NaN,832.135357
8,expat,biologie,18-25,NaN,NaN,1238.646890
9,expat,biologie,25-45,femme,2017.0,122.398329
